In [1]:
! pip -q install transformers

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_size = "small"

# tokenizer = AutoTokenizer.from_pretrained(f"microsoft/DialoGPT-{model_size}")
# model = AutoModelForCausalLM.from_pretrained(f"microsoft/DialoGPT-{model_size}")

In [4]:
def chat(model, tokenizer, trained=False):
    print("type \"q\" to quit. Automatically quits after 5 messages")

    for step in range(5):
        message = input("MESSAGE: ")

        if message in ["", "q"]:  # if the user doesn't wanna talk
            break

        # encode the new user input, add the eos_token and return a tensor in Pytorch
        new_user_input_ids = tokenizer.encode(message + tokenizer.eos_token, return_tensors='pt')

        # append the new user input tokens to the chat history
        bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

        # generated a response while limiting the total chat history to 1000 tokens,
        if (trained):
            chat_history_ids = model.generate(
                bot_input_ids,
                max_length=1000,
                pad_token_id=tokenizer.eos_token_id,
                no_repeat_ngram_size=3,
                do_sample=True,
                top_k=100,
                top_p=0.7,
                temperature = 0.8,
            )
        else:
            chat_history_ids = model.generate(
                bot_input_ids,
                max_length=1000,
                pad_token_id=tokenizer.eos_token_id,
                no_repeat_ngram_size=3
            )

        # pretty print last ouput tokens from bot
        print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

# chat(model, tokenizer)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(f'microsoft/DialoGPT-{model_size}',padding_side='left')

In [7]:
model = AutoModelForCausalLM.from_pretrained(f'/content/drive/MyDrive/DialogPT')

In [9]:
# chat(model, tokenizer, trained=True)

In [10]:
from nltk.translate.bleu_score import sentence_bleu
import torch

def chat_with_bleu_scores(model, tokenizer, questions, expected_responses, trained=False):
    bleu_scores = []

    for question, expected_response in zip(questions, expected_responses):
        print(f"Question: {question}")

        new_user_input_ids = tokenizer.encode(question + tokenizer.eos_token, return_tensors='pt')

        bot_input_ids = new_user_input_ids

        if trained:
            chat_history_ids = model.generate(
                bot_input_ids,
                max_length=1000,
                pad_token_id=tokenizer.eos_token_id,
                no_repeat_ngram_size=3,
                do_sample=True,
                top_k=100,
                top_p=0.7,
                temperature = 0.8,
            )
        else:
            chat_history_ids = model.generate(
                bot_input_ids,
                max_length=1000,
                pad_token_id=tokenizer.eos_token_id,
                no_repeat_ngram_size=3
            )

        generated_response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
        print(f"DialoGPT: {generated_response}")

        reference = [expected_response.split()]
        candidate = generated_response.split()
        bleu_score = sentence_bleu(reference, candidate)
        bleu_scores.append(bleu_score)
        # print(f"BLEU score: {bleu_score}\n")
    return bleu_scores

# Example usage:


In [11]:
questions = [
    "We're going to the park",
    "What are goblins",
    "This is what we're going to do...",
    "Can you hear me?",
    # "Do you miss your family?",
    # "Excuse me Sir"
    # "You’ll stay with me? ",
    # "where have you come from?" ,
    # "What is it with her?"
    ]

expected_responses = [
    "Here he comes, the birthday boy.",
    "goblins but not the most friendly of beasts , have a look at this ",
    "is that when we go out, we're going to buy you two new presents. Come on, why potter “I’m coming with you,”",
    "It's just, I've never talked to a snake before. Oh yeah  ,No i could not  ",
    # "Yes I see. That's me as well. potter yes,i miss",
    # "Rubeus Hagrid, Keeper of Keys and Grounds at Hogwarts., Er — okay, ,,potter, yes right dear" ,
    # "We'll stay with you too. Maybe I’d better stay here,“Let me go!” have they got away space",
    # "a really good question and Me? of course i think this time this drink professor ",
    # "Nothing , she is freak It’s my  friend and she used my wand before and always contains lipstick "
    ]



In [12]:
bleu_scores =chat_with_bleu_scores(model, tokenizer, questions, expected_responses, trained=True)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Question: We're going to the park


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: Here he comes, the birthday boy.
Question: What are goblins


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: Clever as they come goblins but not the most friendly of beasts.
Question: This is what we're going to do...


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: is that when we go out, we're gonna buy you two new presents.
Question: Can you hear me?
DialoGPT: It's just, I've never talked to a snake before.


In [13]:
average_bleu_score = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0
print(f"Average BLEU score across the dataset: {average_bleu_score}")

Average BLEU score across the dataset: 0.5827040227176832
